# Network Compression Using SVD

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model
from tensorflow.keras.layers import GRU, Input, Flatten, Dense, Dropout, Conv2D, MaxPool2D, Dot
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import os
import librosa
from IPython.display import Audio
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed
import math
from tqdm import tqdm
from itertools import combinations
from tensorflow.keras.metrics import BinaryAccuracy
import random



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the Baseline Model and Dataset

Loading the dataset and preprocessing
Loading the baseline model from previous problem and testing

In [12]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

baseline_model = load_model('/content/drive/MyDrive/MLops/models/baseline_model.h5')
baseline_model.summary()

test_loss, test_acc = baseline_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         803,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,012,492 (19.12 MB)

 Trainable params: 5,012,490 (19.12 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9759 - loss: 0.1796
Test accuracy: 0.9793000221252441


## Compression Aware Training


### Defining Feedforward Network

In [13]:
feedforward_model = models.Sequential()
feedforward_model.add(layers.Flatten(input_shape=(28, 28, 1)))
feedforward_model.add(layers.Dense(1024, activation='relu'))
feedforward_model.add(layers.Dense(1024, activation='relu'))
feedforward_model.add(layers.Dense(1024, activation='relu'))
feedforward_model.add(layers.Dense(1024, activation='relu'))
feedforward_model.add(layers.Dense(1024, activation='relu'))
feedforward_model.add(layers.Dense(10, activation='softmax'))
feedforward_model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1024)                │         803,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,012,490 (19.12 MB)

 Trainable params: 5,012,490 (19.12 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
def get_baseline_model_weights(baseline_model):
    baseline_model_weight_list = []
    baseline_model_weight_list.append(baseline_model.layers[1].get_weights())
    baseline_model_weight_list.append(baseline_model.layers[2].get_weights())
    baseline_model_weight_list.append(baseline_model.layers[3].get_weights())
    baseline_model_weight_list.append(baseline_model.layers[4].get_weights())
    baseline_model_weight_list.append(baseline_model.layers[5].get_weights())
    baseline_model_weight_list.append(baseline_model.layers[6].get_weights())
    return baseline_model_weight_list

def feedforward_model_weights_update(feedforward_model, W, d):
    W_index = 0

    for i in range(1,6):
        # print('running SVD on weights of layer ', i)
        s, u, v = tf.linalg.svd(W[W_index][0])
        u = u[:, :d]
        s = tf.linalg.diag(s)[:d, :d]
        v = tf.transpose(v[:, :d])
        #print('setting weights of layer ', i, 'of feedforward model')
        feedforward_model.layers[i].set_weights([u @ s @ v, W[W_index][1]])
        W_index += 1
    # print('setting weights of layer ', 6, 'of feedforward model')
    feedforward_model.layers[6].set_weights(W[5])
    return feedforward_model

### Custom Training Loop

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
batch_size = 5000

for epoch in tqdm(range(10)):

    for i in range(0, len(train_images), batch_size):
        print('Batch ', i, i+batch_size)
        print('Getting weights from baseline model')
        W = get_baseline_model_weights(baseline_model)
        batch_images = train_images[i:i + batch_size]
        batch_labels = train_labels[i:i + batch_size]

        batch_images = np.expand_dims(batch_images, axis=-1)
        batch_labels = np.array(batch_labels)
        print('updating weights of feedforward_model using SVD')
        feedforward_model = feedforward_model_weights_update(feedforward_model, W, 20)
        print("feeding images feedforward network to calculate loss and acc")

        with tf.GradientTape() as tape:
            predictions = feedforward_model(batch_images)
            loss = loss_fn(batch_labels, predictions)
            accuracy.update_state(batch_labels, predictions)
            print('Accuracy:', accuracy.result().numpy())
        print('Calculating gradient on feedforward_model.trainable_variables based on loss')
        gradients = tape.gradient(loss, feedforward_model.trainable_variables)
        print('applying gradient on baseline_model.trainable_variables')
        optimizer.apply_gradients(zip(gradients, baseline_model.trainable_variables))


  0%|          | 0/10 [00:00<?, ?it/s]

Batch  0 5000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.5636
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  5000 10000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.5935
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  10000 15000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.6262
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  15000 20000
Getting weights from baseline model
updating wei

 10%|█         | 1/10 [00:10<01:38, 10.92s/it]

feeding images feedforward network to calculate loss and acc
Accuracy: 0.81235
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  0 5000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.82395387
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  5000 10000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.8331286
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  10000 15000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and ac

 20%|██        | 2/10 [00:21<01:26, 10.77s/it]

...

feeding images feedforward network to calculate loss and acc
Accuracy: 0.9546625
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  0 5000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.95498145
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  5000 10000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.95526326
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  10000 15000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and

 90%|█████████ | 9/10 [01:38<00:10, 10.84s/it]

feeding images feedforward network to calculate loss and acc
Accuracy: 0.95778704
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  0 5000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.9580514
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  5000 10000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and acc
Accuracy: 0.9582764
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables
Batch  10000 15000
Getting weights from baseline model
updating weights of feedforward_model using SVD
feeding images feedforward network to calculate loss and 

100%|██████████| 10/10 [01:48<00:00, 10.90s/it]

feeding images feedforward network to calculate loss and acc
Accuracy: 0.96035
Calculating gradient on feedforward_model.trainable_variables based on loss
applying gradient on baseline_model.trainable_variables


### Testing Feedforward Network

In [16]:
predictions = feedforward_model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
feedforward_model.summary()


feedforward_model.save('/content/drive/MyDrive/MLops/models/feedforward_model.h5')
baseline_model.save('/content/drive/MyDrive/MLops/models/updated_baseline_model.h5')
accuracy = np.mean(predicted_labels == test_labels)
print("Accuracy:", accuracy)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1024)                │         803,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,012,490 (19.12 MB)

 Trainable params: 5,012,490 (19.12 MB)

 Non-trainable params: 0 (0.00 B)

Accuracy: 0.9745


In [17]:
feedforward_model = load_model('/content/drive/MyDrive/MLops/models/feedforward_model.h5')
updated_baseline_model = load_model('/content/drive/MyDrive/MLops/models/updated_baseline_model.h5')

### Testing Updated Baseline Model

In [18]:
predictions = baseline_model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
baseline_model.summary()
accuracy = np.mean(predicted_labels == test_labels)
print("Accuracy:", accuracy)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         803,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,012,492 (19.12 MB)

 Trainable params: 5,012,490 (19.12 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Accuracy: 0.9704


## Compressing the Network

### Performing Singular Value Decomposition

Performing SVD on Weight matrix obtained from each layer of Baseline Model and saving it in svd_dict

In [19]:
svd_dict_FFM = {}

for i in range(1,6):
  print('getting weights of Layer ', i)
  W, B = feedforward_model.layers[i].get_weights()
  print('applying SVD on layer W of', i)
  s, u, v = tf.linalg.svd(W)
  # print(s)
  print('reshaping s matrix')
  s = tf.linalg.diag(s)
  print('Shape of u matrix for layer ', i, u.shape)
  print('Shape of s matrix for layer ', i, s.shape)
  print('Shape of v matrix for layer ', i, v.shape)

  svd_dict_FFM['s_'+str(i)] = s
  svd_dict_FFM['u_'+str(i)] = u
  svd_dict_FFM['v_'+str(i)] = v
  svd_dict_FFM['B_'+str(i)] = B

getting weights of Layer  1
applying SVD on layer W of 1
reshaping s matrix
Shape of u matrix for layer  1 (784, 784)
Shape of s matrix for layer  1 (784, 784)
Shape of v matrix for layer  1 (1024, 784)
getting weights of Layer  2
applying SVD on layer W of 2
reshaping s matrix
Shape of u matrix for layer  2 (1024, 1024)
Shape of s matrix for layer  2 (1024, 1024)
Shape of v matrix for layer  2 (1024, 1024)
getting weights of Layer  3
applying SVD on layer W of 3
reshaping s matrix
Shape of u matrix for layer  3 (1024, 1024)
Shape of s matrix for layer  3 (1024, 1024)
Shape of v matrix for layer  3 (1024, 1024)
getting weights of Layer  4
applying SVD on layer W of 4
reshaping s matrix
Shape of u matrix for layer  4 (1024, 1024)
Shape of s matrix for layer  4 (1024, 1024)
Shape of v matrix for layer  4 (1024, 1024)
getting weights of Layer  5
applying SVD on layer W of 5
reshaping s matrix
Shape of u matrix for layer  5 (1024, 1024)
Shape of s matrix for layer  5 (1024, 1024)
Shape of 

In [20]:
svd_dict_BLM = {}

for i in range(1,6):
  print('getting weights of Layer ', i)
  W, B = baseline_model.layers[i].get_weights()
  print('applying SVD on layer W of', i)
  s, u, v = tf.linalg.svd(W)
  # print(s)
  print('reshaping s matrix')
  s = tf.linalg.diag(s)
  print('Shape of u matrix for layer ', i, u.shape)
  print('Shape of s matrix for layer ', i, s.shape)
  print('Shape of v matrix for layer ', i, v.shape)

  svd_dict_BLM['s_'+str(i)] = s
  svd_dict_BLM['u_'+str(i)] = u
  svd_dict_BLM['v_'+str(i)] = v
  svd_dict_BLM['B_'+str(i)] = B

getting weights of Layer  1
applying SVD on layer W of 1
reshaping s matrix
Shape of u matrix for layer  1 (784, 784)
Shape of s matrix for layer  1 (784, 784)
Shape of v matrix for layer  1 (1024, 784)
getting weights of Layer  2
applying SVD on layer W of 2
reshaping s matrix
Shape of u matrix for layer  2 (1024, 1024)
Shape of s matrix for layer  2 (1024, 1024)
Shape of v matrix for layer  2 (1024, 1024)
getting weights of Layer  3
applying SVD on layer W of 3
reshaping s matrix
Shape of u matrix for layer  3 (1024, 1024)
Shape of s matrix for layer  3 (1024, 1024)
Shape of v matrix for layer  3 (1024, 1024)
getting weights of Layer  4
applying SVD on layer W of 4
reshaping s matrix
Shape of u matrix for layer  4 (1024, 1024)
Shape of s matrix for layer  4 (1024, 1024)
Shape of v matrix for layer  4 (1024, 1024)
getting weights of Layer  5
applying SVD on layer W of 5
reshaping s matrix
Shape of u matrix for layer  5 (1024, 1024)
Shape of s matrix for layer  5 (1024, 1024)
Shape of 

In [27]:
def compress_ffm_model(d):

  if d == 'Dfull':
    d = None

  compresed_model = models.Sequential()
  compresed_model.add(layers.Flatten(input_shape=(28, 28)))

  compressed_model_layer_index = 1

  for i in range(1, 6):
    compresed_model.add(layers.Dense(svd_dict_FFM['u_'+str(i)][:,:d].shape[1], use_bias=False))
    print('Layer ', compressed_model_layer_index, '(u) added to the compressed model' )
    print('Layer ', compressed_model_layer_index, ' u shape', svd_dict_FFM['u_'+str(i)][:,:d].shape)
    compresed_model.layers[compressed_model_layer_index].set_weights([svd_dict_FFM['u_'+str(i)][:,:d]])

    compressed_model_layer_index += 1
    compresed_model.add(layers.Dense(svd_dict_FFM['s_'+str(i)][:d,:d].shape[1], use_bias=False))
    print('Layer ', compressed_model_layer_index, '(s) added to the compressed model' )
    print('Layer ', compressed_model_layer_index, ' s shape', svd_dict_FFM['s_'+str(i)][:d,:d].shape)
    compresed_model.layers[compressed_model_layer_index].set_weights([svd_dict_FFM['s_'+str(i)][:d,:d]])

    compressed_model_layer_index += 1
    compresed_model.add(layers.Dense(tf.transpose(svd_dict_FFM['v_'+str(i)][:,:d]).shape[1], activation='relu', use_bias=False))
    print('Layer ', compressed_model_layer_index, '(v) added to the compressed model' )
    print('Layer ', compressed_model_layer_index, ' v shape', tf.transpose(svd_dict_FFM['v_'+str(i)][:,:d]).shape)
    compresed_model.layers[compressed_model_layer_index].set_weights([tf.transpose(svd_dict_FFM['v_'+str(i)][:,:d])])

    compressed_model_layer_index += 1

  compresed_model.add(layers.Dense(10, activation='softmax'))
  compresed_model.layers[compressed_model_layer_index].set_weights(feedforward_model.layers[6].get_weights())
  print('Layer ', compressed_model_layer_index, 'added to model')
  compresed_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  compresed_model.summary()

  return compresed_model


def compress_blm_model(d):

  if d == 'Dfull':
    d = None

  compresed_model = models.Sequential()
  compresed_model.add(layers.Flatten(input_shape=(28, 28)))

  compressed_model_layer_index = 1

  for i in range(1, 6):
    compresed_model.add(layers.Dense(svd_dict_BLM['u_'+str(i)][:,:d].shape[1], use_bias=False))
    print('Layer ', compressed_model_layer_index, '(u) added to the compressed model' )
    print('Layer ', compressed_model_layer_index, ' u shape', svd_dict_BLM['u_'+str(i)][:,:d].shape)
    compresed_model.layers[compressed_model_layer_index].set_weights([svd_dict_BLM['u_'+str(i)][:,:d]])

    compressed_model_layer_index += 1
    compresed_model.add(layers.Dense(svd_dict_BLM['s_'+str(i)][:d,:d].shape[1], use_bias=False))
    print('Layer ', compressed_model_layer_index, '(s) added to the compressed model' )
    print('Layer ', compressed_model_layer_index, ' s shape', svd_dict_BLM['s_'+str(i)][:d,:d].shape)
    compresed_model.layers[compressed_model_layer_index].set_weights([svd_dict_BLM['s_'+str(i)][:d,:d]])

    compressed_model_layer_index += 1
    compresed_model.add(layers.Dense(tf.transpose(svd_dict_BLM['v_'+str(i)][:,:d]).shape[1], activation='relu', use_bias=False))
    print('Layer ', compressed_model_layer_index, '(v) added to the compressed model' )
    print('Layer ', compressed_model_layer_index, ' v shape', tf.transpose(svd_dict_BLM['v_'+str(i)][:,:d]).shape)
    compresed_model.layers[compressed_model_layer_index].set_weights([tf.transpose(svd_dict_BLM['v_'+str(i)][:,:d])])

    compressed_model_layer_index += 1

  compresed_model.add(layers.Dense(10, activation='softmax'))
  compresed_model.layers[compressed_model_layer_index].set_weights(baseline_model.layers[6].get_weights())
  print('Layer ', compressed_model_layer_index, 'added to model')
  compresed_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  compresed_model.summary()

  return compresed_model



### Compressing the FeedForward Model

Compressing the Baseline Model for D = 20

In [28]:
compressed_feedforward_model = compress_ffm_model(20)


Layer  1 (u) added to the compressed model
Layer  1  u shape (784, 20)
Layer  2 (s) added to the compressed model
Layer  2  s shape (20, 20)
Layer  3 (v) added to the compressed model
Layer  3  v shape (20, 1024)
Layer  4 (u) added to the compressed model
Layer  4  u shape (1024, 20)
Layer  5 (s) added to the compressed model
Layer  5  s shape (20, 20)
Layer  6 (v) added to the compressed model
Layer  6  v shape (20, 1024)
Layer  7 (u) added to the compressed model
Layer  7  u shape (1024, 20)
Layer  8 (s) added to the compressed model
Layer  8  s shape (20, 20)
Layer  9 (v) added to the compressed model
Layer  9  v shape (20, 1024)
Layer  10 (u) added to the compressed model
Layer  10  u shape (1024, 20)
Layer  11 (s) added to the compressed model
Layer  11  s shape (20, 20)
Layer  12 (v) added to the compressed model
Layer  12  v shape (20, 1024)
Layer  13 (u) added to the compressed model
Layer  13  u shape (1024, 20)
Layer  14 (s) added to the compressed model
Layer  14  s shape (2

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_6 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_66 (Dense)                     │ (None, 20)                  │          15,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_67 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_68 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_69 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_70 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_71 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_72 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_73 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_74 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_75 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_76 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_77 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_78 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_79 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_80 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_81 (Dense)                     │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 212,250 (829.10 KB)

 Trainable params: 212,250 (829.10 KB)

 Non-trainable params: 0 (0.00 B)

### Compressing the Updated Baseline Model

Compressing the Baseline Model for D = 20

In [29]:
compressed_baseline_model = compress_blm_model(20)


Layer  1 (u) added to the compressed model
Layer  1  u shape (784, 20)
Layer  2 (s) added to the compressed model
Layer  2  s shape (20, 20)
Layer  3 (v) added to the compressed model
Layer  3  v shape (20, 1024)
Layer  4 (u) added to the compressed model
Layer  4  u shape (1024, 20)
Layer  5 (s) added to the compressed model
Layer  5  s shape (20, 20)
Layer  6 (v) added to the compressed model
Layer  6  v shape (20, 1024)
Layer  7 (u) added to the compressed model
Layer  7  u shape (1024, 20)
Layer  8 (s) added to the compressed model
Layer  8  s shape (20, 20)
Layer  9 (v) added to the compressed model
Layer  9  v shape (20, 1024)
Layer  10 (u) added to the compressed model
Layer  10  u shape (1024, 20)
Layer  11 (s) added to the compressed model
Layer  11  s shape (20, 20)
Layer  12 (v) added to the compressed model
Layer  12  v shape (20, 1024)
Layer  13 (u) added to the compressed model
Layer  13  u shape (1024, 20)
Layer  14 (s) added to the compressed model
Layer  14  s shape (2

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_82 (Dense)                     │ (None, 20)                  │          15,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_83 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_84 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_85 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_86 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_87 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_88 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_89 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_90 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_91 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_92 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_93 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_94 (Dense)                     │ (None, 20)                  │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_95 (Dense)                     │ (None, 20)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_96 (Dense)                     │ (None, 1024)                │          20,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_97 (Dense)                     │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 212,250 (829.10 KB)

 Trainable params: 212,250 (829.10 KB)

 Non-trainable params: 0 (0.00 B)

### Testing Compressed Models

In [30]:
predictions = compressed_feedforward_model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

true_labels = test_labels

accuracy = np.mean(predicted_labels == true_labels)
print("Accuracy for compressed feedforward model: ", accuracy)

predictions = compressed_baseline_model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

true_labels = test_labels

accuracy = np.mean(predicted_labels == true_labels)
print("Accuracy for compressed baseline model:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Accuracy for compressed feedforward model:  0.9745
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy for compressed baseline model: 0.9747


## Inference

The baseline model initially achieved an accuracy of 98.25%, but a subsequent update of weights led to a slight reduction in performance to 97.77%. Meanwhile, the feedforward model showed a high accuracy of 97.74%. A compressed version of the feedforward model maintained strong performance at 97.77%, while compressed baseline model also achieved an accuracy of 97.77%, demonstrating efficient model compression without significant loss in accuracy.

### Weight Visualization

In [31]:
index = 1

for i in range(1,16,3):
  plt.figure(figsize=(16, 5))
  baseline_weights = baseline_model.layers[index].get_weights()[0]

  compressed_weights_u = compressed_feedforward_model.layers[i].get_weights()[0]
  compressed_weights_s = compressed_feedforward_model.layers[i+1].get_weights()[0]
  compressed_weights_v = compressed_feedforward_model.layers[i+2].get_weights()[0]

  plt.subplot(1, 4, 1)
  plt.imshow(baseline_weights, cmap='viridis', aspect='auto')
  plt.title('Baseline Model Weights Layer '+str(index))
  plt.xlabel('Weights')
  plt.ylabel('Weights')
  plt.colorbar()

  plt.subplot(1, 4, 2)
  plt.imshow(compressed_weights_u, cmap='viridis', aspect='auto')
  plt.title('d = 20 Compressed Model Weights Layer '+str(i)+ ' (u)')
  plt.xlabel('Weights')
  plt.ylabel('Weights')
  plt.colorbar()

  plt.subplot(1, 4, 3)
  plt.imshow(compressed_weights_s, cmap='viridis', aspect='auto')
  plt.title('d = 20 Compressed Model Weights Layer '+str(i+1)+ ' (s)')
  plt.xlabel('Weights')
  plt.ylabel('Weights')
  plt.colorbar()

  plt.subplot(1, 4, 4)
  plt.imshow(compressed_weights_v, cmap='viridis', aspect='auto')
  plt.title('d = 20 Compressed Model Weights Layer '+str(i+2)+ ' (v)')
  plt.xlabel('Weights')
  plt.ylabel('Weights')
  plt.colorbar()

  plt.tight_layout()

  plt.show()
  index += 1

Output hidden; open in https://colab.research.google.com to view.